In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Time : ${2023-6-27} ${21:05}
# @Author : Mathew Jin
# @File : ${run_model.py}
# chmod +x run_model.py
# python3 scripts/run_model3.py --process=0 --rawname=1 --resume=0 --anchoring=0
'''
1.TRA
https://arxiv.org/pdf/2106.12950.pdf
https://github.com/microsoft/qlib/blob/main/examples/benchmarks/TRA/src/model.py
1.1 HIST
https://arxiv.org/pdf/2110.13716.pdf
https://github.com/Wentao-Xu/HIST
2.Lightgbm
https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/plot_example.py
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.plot_tree.html
3.other factors
'''
import argparse
import torch
import torch.nn as nn
import numpy as np
import itertools , random , os, shutil , gc , time , h5py , yaml
from tqdm import tqdm
from copy import deepcopy
from torch.optim.swa_utils import AveragedModel , update_bn
from scripts.util.environ import get_logger
from scripts.util.basic import process_timer , FilteredIterator , lr_cosine_scheduler , versatile_storage
from scripts.util.multiloss import multiloss_calculator 
from scripts.data_util.ModelData import ModelData
from scripts.trainer.config import *
from scripts.function.basic import *
from scripts.nn.My import *
# from audtorch.metrics.functional import *

try:
    parser = trainer_parser().parse_args()
except:
    parser = trainer_parser().parse_args(args=[])

TIME_RECODER = process_timer(True)
logger = get_logger()
config = train_config(parser = parser , do_process=True)
set_trainer_configs(config)


usage: ipykernel_launcher.py [-h] [--process PROCESS] [--rawname RAWNAME]
                             [--resume RESUME] [--anchoring ANCHORING]
ipykernel_launcher.py: error: unrecognized arguments: --f=c:\Users\jinmeng\AppData\Roaming\jupyter\runtime\kernel-v2-18836J8KFr0fupm56.json


--What process would you want to run? 0: all, 1: train only (default), 2: test only , 3: copy to instance
--Process Queue : Data + Train + Test + Instance
--Multiple model path of LSTM_day_SHORTTEST exists, input [yes] to resume training, or start a new one!
--Start Training New!
--Multiple model path of LSTM_day_SHORTTEST exists, input [yes] to confirm deletion, or a new directory will be made!
--Directories of [LSTM_day_SHORTTEST] deletion Confirmed!
--Model_name is set to LSTM_day_SHORTTEST!


In [13]:
class process_timer:
    def __init__(self , record = True) -> None:
        self.recording = record
        self.recorder = {} if record else None

    class ptimer:
        def __init__(self , target_dict = None , *args):
            self.target_dict = target_dict
            if self.target_dict is not None:
                self.key = '/'.join(args)
                if self.key not in self.target_dict.keys():
                    self.target_dict[self.key] = []
        def __enter__(self):
            if self.target_dict is not None:
                self.start_time = time.time()
        def __exit__(self, type, value, trace):
            if self.target_dict is not None:
                time_cost = time.time() - self.start_time
                self.target_dict[self.key].append(time_cost)

    def __call__(self , *args):
        return self.ptimer(self.recorder , *args)
    
    def print(self):
        if self.recorder is not None:
            keys = list(self.recorder.keys())
            num_calls = [len(self.recorder[k]) for k in keys]
            total_time = [np.sum(self.recorder[k]) for k in keys]
            tb = pd.DataFrame({'keys':keys , 'num_calls': num_calls, 'total_time': total_time})
            tb['avg_time'] = tb['total_time'] / tb['num_calls']
            print(tb.sort_values(by=['total_time'],ascending=False))

TR = process_timer()
with TR('a/b','c') as t:
    pass
TR.print()


    keys  num_calls  total_time  avg_time
0  a/b/c          1         0.0       0.0


In [9]:
config.update(**new_config(model_path=f'{config.model_base_path}/config_train.yaml'))

AttributeError: 'types.SimpleNamespace' object has no attribute 'update'

In [11]:
SimpleNamespace.__dict__.update(new_config(model_path=f'{config.model_base_path}/config_train.yaml'))

AttributeError: 'mappingproxy' object has no attribute 'update'